In [1]:
import pandas as pd
import numpy as np

# Graph
import plotly.express as px
import matplotlib.pyplot as plt

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras import callbacks

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# tf
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import SGD

In [2]:
path = r"C:\Users\sonia\Documents\Document\2022-2023\Projet 3I - Deep Learning\dataset_clean.csv"
TARGET = 'Label'

In [3]:
df = pd.read_csv(path)

In [4]:
df = df[['Init_Win_bytes_forward',
'Total Length of Fwd Packets',
'Bwd Header Length',
'Destination Port',
'Subflow Fwd Bytes',
'Packet Length Std',
'Packet Length Variance',
'Bwd Packets/s',
'Average Packet Size',
'Bwd Packet Length Std',
         "Label"]]

In [5]:
y = df[TARGET]
X = df.drop([TARGET],axis=1)

In [6]:
y = y.to_numpy()
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
y = to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [8]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X.shape[1]))

In [9]:
model = Sequential()

model.add(GRU(220, input_shape=(1,10), activation="relu", return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(220,activation="relu"))
model.add(Dense(y.shape[1], activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 220)            153120    
                                                                 
 dropout (Dropout)           (None, 1, 220)            0         
                                                                 
 gru_1 (GRU)                 (None, 220)               291720    
                                                                 
 dense (Dense)               (None, 2)                 442       
                                                                 
Total params: 445,282
Trainable params: 445,282
Non-trainable params: 0
_________________________________________________________________


In [10]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", 
                                        patience = 3, 
                                        restore_best_weights = True)

In [11]:
history = model.fit(X_train, y_train, 
                    epochs = 200, 
                    validation_data= (X_test, y_test),
                    callbacks =[earlystopping])
history

Epoch 1/200
619/619 [==============================] - 14s 13ms/step - loss: 0.1698 - accuracy: 0.9509 - val_loss: 0.1364 - val_accuracy: 0.9545
Epoch 2/200
619/619 [==============================] - 7s 12ms/step - loss: 0.1206 - accuracy: 0.9604 - val_loss: 0.1228 - val_accuracy: 0.9580
Epoch 3/200
619/619 [==============================] - 7s 12ms/step - loss: 0.1081 - accuracy: 0.9618 - val_loss: 0.1147 - val_accuracy: 0.9582
Epoch 4/200
619/619 [==============================] - 8s 13ms/step - loss: 0.1013 - accuracy: 0.9631 - val_loss: 0.1088 - val_accuracy: 0.9586
Epoch 5/200
619/619 [==============================] - 7s 11ms/step - loss: 0.0968 - accuracy: 0.9641 - val_loss: 0.1093 - val_accuracy: 0.9580
Epoch 6/200
619/619 [==============================] - 7s 12ms/step - loss: 0.0955 - accuracy: 0.9636 - val_loss: 0.1110 - val_accuracy: 0.9580
Epoch 7/200
619/619 [==============================] - 7s 12ms/step - loss: 0.0944 - accuracy: 0.9635 - val_loss: 0.0986 - val_accuracy

In [12]:
accuracy = model.evaluate(X_test, y_test)
accuracy

155/155 [==============================] - 1s 4ms/step - loss: 0.0946 - accuracy: 0.9630


[0.09460647404193878, 0.9630302786827087]